In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
warnings.filterwarnings('ignore')
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [9]:
#product_form
customer_id = '137'
formatted_attribute = 'Minimum Operating Temperature'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """V-Belt"""

value='%n/a%'
customer_name='%kimballmidwest%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [10]:
df = query_from_file(file_name='./query/curated_products_VBOutDiamMM.sql', params=params)
print(len(df))
print(len(df[df['value'].astype(str)=='n/a']))
df['value'].explode().value_counts()

4106
2833


n/a     2833
-40      613
-67      251
-60      143
-58       51
-65       32
-30       25
14        21
-94       20
-76       17
-25       14
-13       13
0         11
-50       11
-85       10
-49       10
-4         7
-27        7
-14        6
-112       5
15A        1
120        1
-32        1
-45        1
-35        1
221        1
Name: value, dtype: int64

In [11]:
df

attribute value   customer_name external_id  \
0     minimum_operating_temperature   n/a  kimballmidwest       10675   
1     minimum_operating_temperature   n/a  kimballmidwest       22003   
2     minimum_operating_temperature   n/a  kimballmidwest       22004   
3     minimum_operating_temperature   n/a  kimballmidwest       22007   
4     minimum_operating_temperature   n/a  kimballmidwest       22241   
...                             ...   ...             ...         ...   
4101  minimum_operating_temperature     0  kimballmidwest      261584   
4102  minimum_operating_temperature     0  kimballmidwest      261585   
4103  minimum_operating_temperature     0  kimballmidwest      261586   
4104  minimum_operating_temperature     0  kimballmidwest      261587   
4105  minimum_operating_temperature     0  kimballmidwest      261588   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Hex Head SEMS body bolt   
1                                                                                                                                                                                                                                                                                                                                                                                

In [108]:
# regex_pattern = r"(lb)|()"
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df_na=df[(df['value'].astype(str)=='n/a')&(df['matches'].astype(str)!='[]')]#&(df['curated_date'].astype(str)=='2021-06-14')]
# print(len(df_na))
# df_na[0:500]

In [109]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
# list_values.append('mil')
# list_values.append('recy')
# list_values.append('cream')
# list_values.append('onion')

kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['matches']=df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [110]:
list_values

['Cherry',
 'Chocolate',
 'Cognac',
 'Ebony',
 'Espresso',
 'Mahagony',
 'Maple',
 'Mocha',
 'n/a',
 'Natural',
 'Oak',
 'Walnut']

In [111]:
df_na=df[(df['value'].astype(str)=='n/a')]#&(df['resolution'].astype(str)=='rules')]#&(df['curated_date'].astype(str)=='2021-06-14')]
df_na=df_na[df_na['matches'].astype(str)!='[]']
print(len(df_na))
df_na[0:500]

22


attribute value customer_name external_id  \
122  wood_finish   n/a       wbmason  5637449549   
166  wood_finish   n/a       wbmason  5637487916   
194  wood_finish   n/a       wbmason  5637489587   
356  wood_finish   n/a       wbmason  5637508281   
357  wood_finish   n/a       wbmason  5637508282   
365  wood_finish   n/a       wbmason  5637508294   
436  wood_finish   n/a       wbmason  5637514987   
492  wood_finish   n/a       wbmason  5637546253   
497  wood_finish   n/a       wbmason  5637546261   
499  wood_finish   n/a       wbmason  5637546263   
501  wood_finish   n/a       wbmason  5637546297   
530  wood_finish   n/a       wbmason  5637548036   
535  wood_finish   n/a       wbmason  5637548042   
536  wood_finish   n/a       wbmason  5637548043   
537  wood_finish   n/a       wbmason  5637548044   
540  wood_finish   n/a       wbmason  5637548047   
541  wood_finish   n/a       wbmason  5637548048   
564  wood_finish   n/a       wbmason  5637575967   
610  wood_finish   n/a       wbmason  5637587846   
620  wood_finish   n/a       wbmason  5637588129   
621  wood_finish   n/a       wbmason  5637588148   
622  wood_finish   n/a       wbmason  5637588150   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    long_desc  \
122                                                                                                                                                                                                                                                         9200 Series Premium Fabric Upholstered Padded Stack Chair, Ebony Black/Black Santex•Dome-Back Banquet Stack Chair with Plastic Bumpers to Protect from Scratches•2"-Thick High-Density Foam Cushion Built on a MDF Core with Easy-to-Clean Fabric Upholstery•300 lb. Weight Capacity•Strong Square 18-Gauge Steel Frame Plus an Under-Seat H-shaped Brace for Extra Support•Ships Fully Assembled   
166  Fidget Active Seating, Ethylene Vinyl Acetate (EVA) Seat, Black, ABS Plastic, Nylon, Steel, 15.87" Seat Width x 10.50" Seat Depth, 16.3" Width x 13.3" Depth x 18.8" Height•Fidget Seat encourages twisting, rocking and fidgeting for increased muscle engagement throughout the day•Natural fidgeting motion helps encourage increased muscle engagement, focus and activity for movement-rich workday•Seat pan helps to keep the body in a neutral, open-hip position for optimal comfort•Comfortable EVA foam seat cushion features contours for increased comfort and proper usage•Constructed with ABS Plastic and reinforced nylon for durability   
194                                                                                                                                                                                                                                                                                                                                                                                         Student Desk with Open Front Metal Book Box, 24"W x 18"D, Natural•Most preferred student desk with open book box•Designed for plenty of leg room•Spacious 24"W x 18"D work surface•5/8" Thick High Pressure Laminate Top•Laminate top with lacquer edge treatment   
356                                                                                                                                                                                                                                                   

In [112]:
matches=df_na[['external_id']].astype(str)
matches[curation_col]=''
print(len(matches))
matches[0:500]

22


external_id Q:wood_finish
122  5637449549              
166  5637487916              
194  5637489587              
356  5637508281              
357  5637508282              
365  5637508294              
436  5637514987              
492  5637546253              
497  5637546261              
499  5637546263              
501  5637546297              
530  5637548036              
535  5637548042              
536  5637548043              
537  5637548044              
540  5637548047              
541  5637548048              
564  5637575967              
610  5637587846              
620  5637588129              
621  5637588148              
622  5637588150

In [113]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute} - match upload_Shawn.csv',index=False)

In [114]:
looks_good('WBMason', attribute, df, matches)

In [12]:
# df.value.explode().value_counts()

In [13]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [14]:
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# df

In [15]:
df[df['value'].astype(str)=='n/a']

attribute value customer_name external_id  \
12  arm_adjustability   n/a       wbmason  5637159057   
13  arm_adjustability   n/a       wbmason  5637171746   
14  arm_adjustability   n/a       wbmason  5637207276   
15  arm_adjustability   n/a       wbmason  5637212890   
16  arm_adjustability   n/a       wbmason  5637328903   
17  arm_adjustability   n/a       wbmason  5637408149   
18  arm_adjustability   n/a       wbmason  5637455281   
19  arm_adjustability   n/a       wbmason  5637490760   
20  arm_adjustability   n/a       wbmason  5637508240   
21  arm_adjustability   n/a       wbmason  5637541671   
22  arm_adjustability   n/a       wbmason  5637576029   
23  arm_adjustability   n/a       wbmason  5637576091   
24  arm_adjustability   n/a       wbmason  5637576154   

                                                                                                                                                                                                                                                                                                                                                           long_desc  \
12                         2090 Pillow-Soft Series Executive Leather High-Back Swivel/Tilt Chair, Black•Memory foam in seat cushion responds to body dynamics.•Fiber fill in back with recess at the base to cradle the lumbar region.•Tufted, gathered cushions provide exceptional comfort.•Casters supplied with this chair are not suitable for all floor types.   
13                                                   Apprentice II Extended Height Chair, Black Vinyl•Sculpted foam seat and back cushions for superior ergonomic comfort.•Waterfall seat edge may help support better circulation in the legs.•Easy-care upholstery for long wear.•Footring allows legs to rest and may help alleviate pressure points in the legs.   
14                                                                                                       Alera Genaro High-Back Guest Chair, 24.60" x 24.80" x 36.61", Black Seat/Black Back, Black Base•Clean, classic lines complement any office environment.•Contoured foam cushions for all-day comfort.•Padded closed loop arms.•Sturdy black steel sled base.   
15                                                                                                                    Height Adjustable Lab Stool, 13-1/2 dia. x 21h, Black•Easy to clean vinyl upholstery is ideal for medical and education settings.•18 " five-star base and casters offer support and mobility.•Pneumatic lift adapts to different work heights.   
16                                                       R�ve Series Task Chair, Square Plastic Back, Polyester Seat, Black Seat/Back•Contoured, square back provides lumbar support.•Great for any office workspace or home office.•Perforated back provides cool all day comfort.•Chair is accented by silver frame with five-star base with casters for mobility.   
17                  Alera Sorrento Series Ultra-Cushioned Stacking Guest Chair, Supports Up to 275 lb, Black, 2/Carton•Multi-purpose chair with black PVC-free faux leather upholstery.•Ultra-cushioned seat and back for enhanced comfort.•Perfect for waiting areas, conference rooms or guest seating.•Steel base with straight legs for strength and durability.   
18                                              Convergence Chair, Black•Black mesh back is breathable for all-day comfort and support.•Black seat fabric flexes and stretches to make the soft foam even more responsive.•Thick seat cushions to take the stress off the lower body throughout the day.•Lumbar support that adds a little extra lower back comfort.   
19                                                                                                                                                               Mobile Wooden Ergonomic Kneeling Chair in Black Fabric•Contemporary style•Padded seat and knee rest•CA117 Fire Retardant Foam•Knee rest pad: 17.75"W x 9.75"D x 2.75" thick•

In [16]:
df[df['value'].astype(str)=='n/a'][100:150]

Empty DataFrame
Columns: [attribute, value, customer_name, external_id, long_desc, curated_date, resolution, matches]
Index: []

In [17]:
df[df['external_id'].astype(str)=='00739441']

Empty DataFrame
Columns: [attribute, value, customer_name, external_id, long_desc, curated_date, resolution, matches]
Index: []

In [18]:
x['matches'].explode().value_counts()[50:100]

NameError: name 'x' is not defined

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [ ]:
split_col(x)

In [ ]:
len(x['matches'].explode().value_counts())

In [ ]:
# error

In [ ]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
x

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
#         numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

In [ ]:
x['matches']=x['matches'].astype(float)

In [ ]:
x

# matches

In [ ]:
matches=x
matches

In [ ]:
matches['matches']=matches['matches'].astype(float)

In [ ]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)

In [ ]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

In [ ]:
matches[0:50]

In [ ]:
matches['Q:micro_v_belt_overall_length'].explode().value_counts()

# fixed_na

In [ ]:
fixed_na=matches[matches['Q:micro_v_belt_overall_length'].astype(str)=='n/a']

In [ ]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [ ]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

In [ ]:
fixed_na['Q:micro_v_belt_overall_length'].explode().value_counts()

# Wipe

In [ ]:
wipe=df[['external_id']].astype(str)
wipe['Q:micro_v_belt_overall_length']=''
wipe

# Null Values

In [ ]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [ ]:
na_external_id

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [ ]:
looks_good('WBMason', attribute, df, matches)

In [ ]:
matches